In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np

pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
ds = pd.read_csv('./Raw_V1.csv')
ds.head()

,StartDate,EndDate,GeoID,Geo,CategoryID,Category,Impressions
0,01/01/2016,31/01/2016,16632,Israel,11111,Nail Fungus Products,300366
1,01/01/2016,31/01/2016,16632,Israel,22222,Key Making & Copying,13333338
2,01/01/2016,31/01/2016,16632,Israel,33333,TV Sci-Fi & Fantasy Shows,51466
3,01/01/2016,31/01/2016,19782,United Kingdom,11111,Nail Fungus Products,725217
4,01/01/2016,31/01/2016,19782,United Kingdom,22222,Key Making & Copying,783398


In [3]:
ds['month'] = ds.StartDate.map(lambda x: x.split('/')[1])
ds['year'] = ds.StartDate.map(lambda x: x.split('/')[2])
ds.head()

,StartDate,EndDate,GeoID,Geo,CategoryID,Category,Impressions,month,year
0,01/01/2016,31/01/2016,16632,Israel,11111,Nail Fungus Products,300366,01,2016
1,01/01/2016,31/01/2016,16632,Israel,22222,Key Making & Copying,13333338,01,2016
2,01/01/2016,31/01/2016,16632,Israel,33333,TV Sci-Fi & Fantasy Shows,51466,01,2016
3,01/01/2016,31/01/2016,19782,United Kingdom,11111,Nail Fungus Products,725217,01,2016
4,01/01/2016,31/01/2016,19782,United Kingdom,22222,Key Making & Copying,783398,01,2016


In [4]:
geoDf = ds.filter(items=['GeoID','Geo']).drop_duplicates()
categoryDf = ds.filter(items=['CategoryID','Category']).drop_duplicates()
yearsDf = ds.filter(items=['year']).drop_duplicates()

print(geoDf.head())
print(categoryDf.head())
print(yearsDf.head())

   GeoID             Geo
0  16632          Israel
3  19782  United Kingdom
6  19880   United States
   CategoryID                   Category
0       11111       Nail Fungus Products
1       22222       Key Making & Copying
2       33333  TV Sci-Fi & Fantasy Shows
     year
0    2016
108  2017
216  2018


In [5]:
columnsNames = ['CategoryID', *yearsDf.year.array]
peakMonth = pd.DataFrame([], columns=['GeoID', *columnsNames])
peakMonth.head()

,GeoID,CategoryID,2016,2017,2018


In [6]:
val = [0]*len(yearsDf.year.array)

for geo in geoDf.GeoID:
    masterdf = []
    for cat in categoryDf.CategoryID:
        data = [cat]
        data.extend(val)
        tempdf = {'CategoryID':cat, 'GeoID':geo}
        for year in yearsDf.year:
            temp = ds[(geo==ds.GeoID) & (cat==ds.CategoryID) & (year == ds.year)]
            temp = temp.filter(items=['Impressions','month'])
            maxImpressionsId = temp.Impressions.idxmax(axis=0)
            tempdf.update([(year, temp[temp.index == maxImpressionsId].month.values[0])])
        masterdf.append(tempdf)
    peakMonth = peakMonth.append(masterdf, ignore_index=True)

In [7]:
print(peakMonth)

   GeoID CategoryID 2016 2017 2018
0  16632      11111   03   02   11
1  16632      22222   01   01   12
2  16632      33333   12   12   12
3  19782      11111   06   06   06
4  19782      22222   08   03   01
5  19782      33333   12   11   02
6  19880      11111   08   06   03
7  19880      22222   01   01   01
8  19880      33333   06   11   01


In [8]:
peakMonth= peakMonth.join(geoDf.set_index('GeoID'), on='GeoID')
peakMonth= peakMonth.join(categoryDf.set_index('CategoryID'), on='CategoryID')
print(peakMonth)

   GeoID CategoryID 2016 2017 2018             Geo                   Category
0  16632      11111   03   02   11          Israel       Nail Fungus Products
1  16632      22222   01   01   12          Israel       Key Making & Copying
2  16632      33333   12   12   12          Israel  TV Sci-Fi & Fantasy Shows
3  19782      11111   06   06   06  United Kingdom       Nail Fungus Products
4  19782      22222   08   03   01  United Kingdom       Key Making & Copying
5  19782      33333   12   11   02  United Kingdom  TV Sci-Fi & Fantasy Shows
6  19880      11111   08   06   03   United States       Nail Fungus Products
7  19880      22222   01   01   01   United States       Key Making & Copying
8  19880      33333   06   11   01   United States  TV Sci-Fi & Fantasy Shows


In [9]:
def is_peak(geo, category):
    geo_category_filtered_peak = peakMonth[(peakMonth.GeoID==geo) & (peakMonth.CategoryID==category)]
    return geo_category_filtered_peak.filter(items=yearsDf.year).nunique(axis=1) == 1

In [10]:
peak_arr = []
for idx, row in peakMonth.iterrows():
    peak_arr.extend(is_peak(row['GeoID'], row['CategoryID']))
peakMonth.insert(len(peakMonth.columns)-1, 'peak', peak_arr)

print(peak_arr)

[False, False, True, True, False, False, False, True, False]


In [18]:
#find only peaks
onlyPeaks = peakMonth[peakMonth.peak== True]
print(onlyPeaks)


   GeoID CategoryID 2016 2017 2018             Geo  peak  \
2  16632      33333   12   12   12          Israel  True   
3  19782      11111   06   06   06  United Kingdom  True   
7  19880      22222   01   01   01   United States  True   

                    Category  
2  TV Sci-Fi & Fantasy Shows  
3       Nail Fungus Products  
7       Key Making & Copying  


In [19]:
## find the average impressions of vertical in all months 
totalAvgs = ds.groupby(['CategoryID', 'GeoID'], as_index=False, sort=False).mean()
totalAvgs.rename(columns={'Impressions':'TotalImpressionsAvg'},inplace=True)
print(totalAvgs)


   CategoryID  GeoID  TotalImpressionsAvg
0       11111  16632            438432.42
1       22222  16632            852240.67
2       33333  16632            497760.72
3       11111  19782            702651.64
4       22222  19782            549852.75
5       33333  19782            481776.69
6       11111  19880            516305.64
7       22222  19880           1551682.69
8       33333  19880            503668.86


In [20]:
## function to find the average impressions of the peak month over all of the years 
peakAvg = pd.DataFrame()
for idx, row in onlyPeaks.iterrows():
    peakAvg = peakAvg.append(ds[(ds.GeoID==row.GeoID) & (ds.CategoryID==row.CategoryID) & (ds.month==row['2018'])].groupby(['CategoryID', 'GeoID', 'month'], as_index=False, sort=False).mean())
peakAvg.rename(columns={'Impressions':'PeakImpressionsAvg', 'month':'PeakMonth'},inplace=True)
print(peakAvg)


   CategoryID  GeoID PeakMonth  PeakImpressionsAvg
0       33333  16632        12             1001001
0       11111  19782        06             2002002
0       22222  19880        01            13333338


In [21]:
merged = pd.merge(peakAvg, totalAvgs, on=['GeoID', 'CategoryID'])
merged['ratio'] = merged['PeakImpressionsAvg'] / merged['TotalImpressionsAvg']
merged= merged.join(geoDf.set_index('GeoID'), on='GeoID')
merged= merged.join(categoryDf.set_index('CategoryID'), on='CategoryID')
merged = merged.sort_values('ratio', ascending=False)
merged

,CategoryID,GeoID,PeakMonth,PeakImpressionsAvg,TotalImpressionsAvg,ratio,Geo,Category
2,22222,19880,01,13333338,1551682.69,8.59,United States,Key Making & Copying
1,11111,19782,06,2002002,702651.64,2.85,United Kingdom,Nail Fungus Products
0,33333,16632,12,1001001,497760.72,2.01,Israel,TV Sci-Fi & Fantasy Shows


In [15]:
merged.to_csv('./result_V1.csv')